# Object Tracking

Detector: Multi Class Pre Trained YOLO v3

Rastreador: SORT (Simple, online, real-time detector)

In [1]:
# Importar Bibliotecas
from models import *
from utils import *

import os, sys, time, datetime, random, shutil, glob
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.autograd import Variable

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

import cv2
from IPython.display import clear_output
from sort import *

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\linear_assignment_.py:22: FutureWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


In [2]:
config_path  = 'config/yolov3.cfg'
weights_path = 'config/yolov3.weights'
class_path   = 'config/coco.names'
img_size     = 416
conf_thres   = 0.8
nms_thres    = 0.4

# Load model and weights
model = Darknet(config_path, img_size=img_size)
model.load_weights(weights_path)
model.eval()
classes = utils.load_classes(class_path)
Tensor  = torch.FloatTensor

C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [3]:
def detect_image(img):
    # scale and pad image
    ratio = min(img_size/img.size[0], img_size/img.size[1])
    imw = round(img.size[0] * ratio)
    imh = round(img.size[1] * ratio)
    img_transforms = transforms.Compose([ transforms.Resize((imh, imw)),
         transforms.Pad((max(int((imh-imw)/2),0), max(int((imw-imh)/2),0), max(int((imh-imw)/2),0), max(int((imw-imh)/2),0)),
                        (128,128,128)),
         transforms.ToTensor(),
         ])
    # convert image to Tensor
    image_tensor = img_transforms(img).float()
    image_tensor = image_tensor.unsqueeze_(0)
    input_img = Variable(image_tensor.type(Tensor))
    # run inference on the model and get detections
    with torch.no_grad():
        detections = model(input_img)
        detections = utils.non_max_suppression(detections, 80, conf_thres, nms_thres)
    return detections[0]

In [4]:
def analisar_video(folderpath, filename):
    global resulting_video_path
    videopath = folderpath+filename
    dirName = "object-detection-video-"+filename
    resulting_video_path = ".\\" + dirName[:-4]
    print(resulting_video_path)
    
    if not os.path.exists(resulting_video_path):
        os.mkdir(resulting_video_path)
        print("Directory " , dirName ,  " Created ")
    else:
        shutil.rmtree(resulting_video_path)
        os.mkdir(resulting_video_path)
        print("Directory " , dirName ,  " Created ")
        #print("Directory " , dirName ,  " already exists")
    
    cmap = plt.get_cmap('tab20b')
    colors = [cmap(i)[:3] for i in np.linspace(0, 1, 20)]

    # initialize Sort object and video capture
    vid         = cv2.VideoCapture(videopath)
    mot_tracker = Sort() 
    
    now = datetime.datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("Começo do processamento: ", current_time, "\n")
    
    # Video Properties
    vid_time_milisseconds = 0
    vid_total_frames = vid.get(cv2.CAP_PROP_FRAME_COUNT)
    print("Total de Frames do Vídeo: ", vid_total_frames)
    
    frame_atual = 0
    while(True):
        vid.set(cv2.CAP_PROP_POS_MSEC, vid_time_milisseconds)
        vid_time_milisseconds += 100
        succes_reading_video, frame = vid.read()
        
        if not succes_reading_video or frame_atual > 50:
            now = datetime.datetime.now()
            current_time = now.strftime("%H:%M:%S")
            print("Fim do Processamento: ", current_time)
            break

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_atual += 1
        pilimg = Image.fromarray(frame)
        detections = detect_image(pilimg)

        img = np.array(pilimg)
        pad_x = max(img.shape[0] - img.shape[1], 0) * (img_size / max(img.shape))
        pad_y = max(img.shape[1] - img.shape[0], 0) * (img_size / max(img.shape))
        unpad_h = img_size - pad_y
        unpad_w = img_size - pad_x
        if detections is not None:
            tracked_objects = mot_tracker.update(detections.cpu())

            unique_labels = detections[:, -1].cpu().unique()
            n_cls_preds = len(unique_labels)
            for x1, y1, x2, y2, obj_id, cls_pred in tracked_objects:
                box_h = int(((y2 - y1) / unpad_h) * img.shape[0])
                box_w = int(((x2 - x1) / unpad_w) * img.shape[1])
                y1 = int(((y1 - pad_y // 2) / unpad_h) * img.shape[0])
                x1 = int(((x1 - pad_x // 2) / unpad_w) * img.shape[1])

                color = colors[int(obj_id) % len(colors)]
                color = [i * 255 for i in color]
                cls = classes[int(cls_pred)]
                cv2.rectangle(frame, (x1, y1), (x1+box_w, y1+box_h), color, 4)
                cv2.rectangle(frame, (x1, y1-35), (x1+len(cls)*19+60, y1), color, -1)
                cv2.putText(frame, cls + "-" + str(int(obj_id)), (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 3)
        print(resulting_video_path+"\\"+str(frame_atual)+str(".jpg"))
        cv2.imwrite(resulting_video_path+"\\"+str(frame_atual)+str(".jpg"), frame)
        #fig = plt.figure(figsize=(15, 15))
        #ax = fig.add_subplot(1, 1, 1)
        #ax.set_title("Video Stream")
        #ax.imshow(frame)
        #plt.show()
        #clear_output(wait=True)
    return resulting_video_path

folder_path = r'C:\Users\Ribeiro\Google Drive\Mestrado\Dissertação\Vídeos\\'
videopath   = r'production_ID_3694423_1280x720_approx30fps.mp4'

resulting_video_path = analisar_video(folder_path,videopath)

.\object-detection-video-production_ID_3694423_1280x720_approx30fps
Directory  object-detection-video-production_ID_3694423_1280x720_approx30fps.mp4  Created 
Começo do processamento:  12:09:32 

Total de Frames do Vídeo:  1905.0
.\object-detection-video-production_ID_3694423_1280x720_approx30fps\1.jpg


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


.\object-detection-video-production_ID_3694423_1280x720_approx30fps\2.jpg
.\object-detection-video-production_ID_3694423_1280x720_approx30fps\3.jpg
.\object-detection-video-production_ID_3694423_1280x720_approx30fps\4.jpg
.\object-detection-video-production_ID_3694423_1280x720_approx30fps\5.jpg
.\object-detection-video-production_ID_3694423_1280x720_approx30fps\6.jpg
.\object-detection-video-production_ID_3694423_1280x720_approx30fps\7.jpg
.\object-detection-video-production_ID_3694423_1280x720_approx30fps\8.jpg
.\object-detection-video-production_ID_3694423_1280x720_approx30fps\9.jpg
.\object-detection-video-production_ID_3694423_1280x720_approx30fps\10.jpg
.\object-detection-video-production_ID_3694423_1280x720_approx30fps\11.jpg
.\object-detection-video-production_ID_3694423_1280x720_approx30fps\12.jpg
.\object-detection-video-production_ID_3694423_1280x720_approx30fps\13.jpg
.\object-detection-video-production_ID_3694423_1280x720_approx30fps\14.jpg
.\object-detection-video-producti

In [ ]:
# Criar Vídeo Na Nova Pasta
